# Start a async sentiment analysis
Input objects dataframe see `docs/schemas/object.md`  

Ouput an AsyncJobGroup that can be used to complete the sentiment analysis.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag.third_party_models.aws_async import start_sentiment
from phoenix.tag.third_party_models.aws_async import job_types

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

# TODO: this has to be refactored so that MONTH_PREFIX is relational to the YEAR_FILTER and MONTH_FILTER
YEAR_FILTER = 2021
MONTH_FILTER = 5
MONTH_PREFIX = f"year_filter={YEAR_FILTER}/month_filter={MONTH_FILTER}/"

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}tagging_runs/{MONTH_PREFIX}facebook_posts/"
ARTIFACTS_COMPREHEND_BASE_URL = "s3://bu-undp-comprehend-from-local/"

# Input
OBJECTS_PATH = artifacts.dataframes.url(ARTIFACTS_BASE_URL, "objects")

# Output
ASYNC_JOB_GROUP = artifacts.json.url(ARTIFACTS_BASE_URL, "async_job_group")

# DataAccessRole
AWS_DATA_ACCESS_ROLE = "arn:aws:iam::018241716617:role/AWSServiceComprehend"

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
ARTIFACTS_COMPREHEND_BASE_URL,
RUN_DATE,
ASYNC_JOB_GROUP,
AWS_DATA_ACCESS_ROLE,
sep='\n',
)

In [ ]:
objects = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "objects")).dataframe

In [ ]:
objects.head()

In [ ]:
async_job_group = start_sentiment.start_sentiment_analysis_jobs(AWS_DATA_ACCESS_ROLE, ARTIFACTS_COMPREHEND_BASE_URL, objects)

In [ ]:
async_job_group

In [ ]:
_ = job_types.persist_json(ASYNC_JOB_GROUP, async_job_group)